# Background LCIA

This notebook calculates the indirect impacts using pre-calculated impact intensities with the LCIA of each product.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#In this cell the scenario is chosen. This can either be defined here or in the RUN_ALL.ipynb
#If defined here unmark the scen = line and select the scenario you will run.
#If defined in RUN_ALL.ipynb unmark the %store -r scen line.
#Scenario names are:
#S0_MinFert > N,P,K demand met by mineral fertilizers only
#S0_struvite_P > P demand met by struvite, N demand met by struvite and mineral fertilizer and K demand met by mineral fertilizer
#S0_compost > N,P,K supplied by compost produced in the AMB, remaining N,P,K demand met by mineral fertilizer
#S0_Ammon_salts > N supplied from recovered ammonium salts and from mineral fertilizers, P from struvite and K from mineral fertilizer
scen = 'S0_MinFert'
#%store -r scen

In [ ]:
print(scen)

In [ ]:
%store -r URBAG_map
%store -r Nitrogen_Balance
%store -r Phosphorus_Balance
%store -r Potassium_Balance
%store -r Mineral_fert_K2O

In [ ]:
URBAG_map.columns.values

In [ ]:
URBAG_map = URBAG_map.drop(['TIPO_RIEGO', 'EFIC_RIEGO', 'TIPO_DIST', 'EFIC_DISTR', 'DUN_GRUP',
       'DUN_CULTIU', 'TF', 'REG_CREAF', 'REG_CAT', 'TF_REG',
        'CAT_NIV_4', 'ORD_NIV_4', 'CAT_NIV_3',
       'ORD_NIV_3', 'CAT_NIV_2', 'ORD_NIV_2', 'CAT_NIV_1F', 'ORD_NIV_1F',
       'CAT_NIV_1', 'ORD_NIV_1', '01_ETO', '02_ETO', '03_ETO', '04_ETO',
       '05_ETO', '06_ETO', '07_ETO', '08_ETO', '09_ETO', '10_ETO',
       '11_ETO', '12_ETO', '01_PPT', '02_PPT', '03_PPT', '04_PPT',
       '05_PPT', '06_PPT', '07_PPT', '08_PPT', '09_PPT', '10_PPT',
       '11_PPT', '12_PPT', 'ID_Voronoi', 'Municipi', 'Codi_munic',
       'Codi_comar', 'ID_CUENCA', 'CUENCA',
       'ID_SUBCUEN', 'N_SUBCUENC', 'SUBCUENCAS',
       'yield_kgHA', 'kgP2O5/ha', 'kgK2O/ha', 'kgN/ha', 'EF_Tier2',
       'id_final', 'Kc_b_Kc1', 'Kc_b_Kc2', 'Kc_b_Kc3', 'Kc_b_Kc4',
       'Kc_b_Kc5', 'Kc_b_Kc6', 'Kc_b_Kc7', 'Kc_b_Kc8', 'Kc_b_Kc9',
       'Kc_b_Kc10', 'Kc_b_Kc11', 'Kc_b_Kc12', 'N_T', 'P_T', 'K_T',
       'Voronoi_ag', 'Vor_ag_Num', '01_CWR', '02_CWR',
       '03_CWR', '04_CWR', '05_CWR', '06_CWR', '07_CWR', '08_CWR',
       '09_CWR', '10_CWR', '11_CWR', '12_CWR', '01_PPTe', '02_PPTe',
       '03_PPTe', '04_PPTe', '05_PPTe', '06_PPTe', '07_PPTe', '08_PPTe',
       '09_PPTe', '10_PPTe', '11_PPTe', '12_PPTe', '01_ID', '02_ID',
       '03_ID', '04_ID', '05_ID', '06_ID', '07_ID', '08_ID', '09_ID',
       '10_ID', '11_ID', '12_ID', 'LOSSES', '01_WE', '02_WE', '03_WE',
       '04_WE', '05_WE', '06_WE', '07_WE', '08_WE', '09_WE', '10_WE',
       '11_WE', '12_WE', 'WE_m3_ha_yr', 'Manure', 'Manure_N',
       'Manure_P2O5', 'Agriwaste_toField', 'Agriwaste_burned',
       'N_agriwaste_to_field', 'Cuencas',
       'GWP100_IPCC_N2O', 'GWP100_IPCC_CO2', 'GWP100_IPCC_CH4', 'ME_NO3',
       'ME_NO3_Reg', 'FE_PO43', 'FE_PO43_Reg', 'WC'], axis=1)

## LCIA Methods: Family and version, Name, Unit, # of CFsS

- IPCC 2021, climate change, global warming potential (GWP100)	kg CO2-Eq	226  
- ReCiPe 2016 v1.03, midpoint (H), eutrophication: marine, marine eutrophication potential (MEP)	kg N-Eq	22 
- ReCiPe 2016 v1.03, midpoint (H), eutrophication: freshwater, freshwater eutrophication potential (FEP)	kg P-Eq	13 
- ReCiPe 2016 v1.03, midpoint (H), water use, water consumption potential (WCP) m3 of water consumed/m3 water extracted 5 
- CML v4.8 2016, material resources: metals/minerals, abiotic depletion potential (ADP): elements (ultimate reserves)	kg Sb-Eq	129

In [ ]:
Phosphorus_Balance.keys()

In [ ]:
Nitrogen_Balance.keys()

In [ ]:
Potassium_Balance.keys()

## Pre-calculated impact intensities per product

In [ ]:
#Impact of production of each nutrient supply option in kg impact units per kg of product e.g. kg CO2eq/kg MAP
back_impacts = pd.read_excel("inputs/URBAG_LCIAs_ei391cutoff_20240409.xlsx")

In [ ]:
back_impacts.keys()

In [ ]:
pd.unique(back_impacts['database'])

In [ ]:
pd.unique(back_impacts['reference product'])

In [ ]:
back_impacts['database'][back_impacts['database'].str.fullmatch('ei391_cutoff_circular_strategies')]                       = "ei391_cutoff"
back_impacts['database'][back_impacts['database'].str.fullmatch('image - SSP2-Base - 2050')]                               = "ei391_cutoff_image_SSP2_Base_2050"
back_impacts['database'][back_impacts['database'].str.fullmatch('ei391_cutoff_CS_image_SSP2-Base-2050')]                   = "ei391_cutoff_image_SSP2_Base_2050"
back_impacts['database'][back_impacts['database'].str.fullmatch('image - SSP2-RCP26 - 2050')]                              = "ei391_cutoff_image_SSP2_RCP26_2050"
back_impacts['database'][back_impacts['database'].str.fullmatch('ei391_cutoff_CS_image_SSP2-RCP26-2050')]                  = "ei391_cutoff_image_SSP2_RCP26_2050"

In [ ]:
pd.unique(back_impacts['database'])

In [ ]:
ReCiPe_LCIA_1 = back_impacts.drop(['Unnamed: 0',"index"],axis=1)

In [ ]:
ReCiPe_LCIA = pd.melt(ReCiPe_LCIA_1, id_vars=['amount', 'unit', 'reference product', 'name', 'location',
       'database'], value_vars=[
       'ReCiPe 2016 v1.03, midpoint (H) | eutrophication: freshwater | freshwater eutrophication potential (FEP)',
       'ReCiPe 2016 v1.03, midpoint (H) | eutrophication: marine | marine eutrophication potential (MEP)',
       'ReCiPe 2016 v1.03, midpoint (H) | water use | water consumption potential (WCP)',
       'IPCC 2021 | climate change | global warming potential (GWP100)',
       'CML v4.8 2016 | material resources: metals/minerals | abiotic depletion potential (ADP): elements (ultimate reserves)'],var_name='impact')

In [ ]:
ReCiPe_LCIA

In [ ]:
pd.unique(ReCiPe_LCIA['database'])

In [ ]:
#Background databases [3 databases]> ei, baseline2050, RCP2.62050, for SSP2
databases = pd.unique(ReCiPe_LCIA['database'])
len(databases)

In [ ]:
#Impact categories [5 impacts: ADP, FE, ME, GWP, WC]
impacts = pd.unique(ReCiPe_LCIA['impact'])
len(impacts)

In [ ]:
#Products [6 products]> N, P, K mineral fertilizer and struvite with biogas co-generation, Ostara Llobregat WWTP, compost from market AMB (3 composting plants), ammonium salts+struvite
products = pd.unique(ReCiPe_LCIA['reference product'])
len(products)

In [ ]:
Nitrogen_Balance.keys()

In [ ]:
Phosphorus_Balance.keys()

In [ ]:
Potassium_Balance.keys()

## Content of N and P in struvite, compost and ammonium salts

This constants are used to convert from N to kg of products and viceversa

### Struvite

    - According to Want et.al. (2023)

        *kg P2O5 per kg MAP = 0.25
        *kg N per kg MAP = 0.045
    
    - ICTA struvite (used here)
        
        *kg P2O5 per kg MAP = 0.29
        *kg N per kg MAP = 0.057

In [ ]:
kg_P2O5_kg_MAP = 0.29
kg_MAP_kg_P2O5 = 1/0.29
kg_N_kg_MAP = 0.057
kg_MAP_kg_N = 1/0.057

### Compost

According to data for three composting plants considered:
    
        *Sant Cugat
        *Torrelles
        *Zona Franca

In [ ]:
#P content per compost plant * bioavailability * convesion from P to P2O5
#kg P2O5 per kg compost FM
kg_P2O5_kg_FMcompost_SantCugatCP   = 0.0057*0.95*(1/0.4364)
kg_P2O5_kg_FMcompost_TorrellesCP   = 0.0064*0.95*(1/0.4364)
kg_P2O5_kg_FMcompost_ZonaFrancaECP = 0.0197*0.95*(1/0.4364)

In [ ]:
kg_DMcompost_kg_FMcompost_SantCugatCP = 0.789
kg_DMcompost_kg_FMcompost_TorrellesCP = 0.779
kg_DMcompost_kg_FMcompost_ZonaFrancaECP = 0.592

In [ ]:
#Average nutrient content for the compost of the three composting plants considered for the AMB
kg_compostFM_kg_P2O5 = 1/np.average([(kg_P2O5_kg_FMcompost_SantCugatCP),(kg_P2O5_kg_FMcompost_TorrellesCP),(kg_P2O5_kg_FMcompost_ZonaFrancaECP)], weights=[0.21,0.15,0.64])
kg_compostFM_kg_P2O5

In [ ]:
#Average moisture content for the compost of the three composting plants considered for the AMB
kg_DMcompost_kg_FMcompost = 1/np.average([(kg_DMcompost_kg_FMcompost_SantCugatCP),(kg_DMcompost_kg_FMcompost_TorrellesCP),(kg_DMcompost_kg_FMcompost_ZonaFrancaECP)], weights=[0.21,0.15,0.64])
kg_DMcompost_kg_FMcompost

### Ammonium salts

Ammonium salts content of Nitrogen is 18.8%, according to ENRICH pilot data (Mayor et.al, 2023)

In [ ]:
kg_AmmoniumSalt_kg_N = 1/0.188
kg_AmmoniumSalt_kg_N

## Fertilizer, struvite, compost and ammonium salts use

In [ ]:
activity = pd.DataFrame(URBAG_map)

In [ ]:
activity.keys()

In [ ]:
activity["N_Fertilizer"]     = Nitrogen_Balance['N_Fertiliser']                #kg N/ha yr
activity["P_Fertilizer"]     = Phosphorus_Balance['P_Fertiliser']              #kg P2O5/ha yr
activity["K_Fertilizer"]     = Potassium_Balance['K_Fertiliser']               #kg K2O/ ha yr

In [ ]:
activity["N_Ammonium_salts"] = Nitrogen_Balance['N_Ammonium_salts'] *kg_AmmoniumSalt_kg_N #kg N/ha yr * kg Ammonim salts/kg N = kg Ammonium salts/ha yr

In [ ]:
#Struvite use in each scenario in kg MAP per ha yr
#we use the ammount determined in the P balance as this is the limiting nutrient and in fact only struvite is used in the struvite_P scenario
if scen == 'S0_MinFert':    
    activity["Struvite"]  = Phosphorus_Balance['P_Struvite']*kg_MAP_kg_P2O5 #kg P2O5/ha yr * kg MAP/kg P2O5 = kg MAP/ha yr [should be zeros!!]
elif scen == 'S0_struvite_P':
    activity["Struvite"]  = Phosphorus_Balance['P_Struvite']*kg_MAP_kg_P2O5 #kg P2O5/ha yr * kg MAP/kg P2O5 = kg MAP/ha yr
elif scen == 'S0_compost':
    activity["Struvite"]  = Phosphorus_Balance['P_Struvite']*kg_MAP_kg_P2O5 #kg P2O5/ha yr * kg MAP/kg P2O5 = kg MAP/ha yr [should be zeros!!]
elif scen == 'S0_Ammon_salts':
    activity["Struvite"]  = Phosphorus_Balance['P_Struvite']*kg_MAP_kg_P2O5 #kg P2O5/ha yr * kg MAP/kg P2O5 = kg MAP/ha yr
else:
    pass

In [ ]:
#Compost use in each scenario in kg FM compost per ha yr
#We use the ammount determined in the P balance as this is the nutrient more supplied with compost and in fact only compost is used in the compost scenario
if scen == 'S0_MinFert':    
    activity["Compost"]  = Phosphorus_Balance['P_Compost']*kg_compostFM_kg_P2O5*kg_DMcompost_kg_FMcompost #kg P2O5/ha yr * kg DMcompost/kg P2O5 *kg FM compost/kg DM compost = kg FM compost/ha yr [should be zeros!!]
elif scen == 'S0_struvite_P':
    activity["Compost"]  = Phosphorus_Balance['P_Compost']*kg_compostFM_kg_P2O5*kg_DMcompost_kg_FMcompost #kg P2O5/ha yr * kg DMcompost/kg P2O5 *kg FM compost/kg DM compost = kg FM compost/ha yr [should be zeros!!]
elif scen == 'S0_compost':
    activity["Compost"]  = Phosphorus_Balance['P_Compost']*kg_compostFM_kg_P2O5*kg_DMcompost_kg_FMcompost #kg P2O5/ha yr * kg DMcompost/kg P2O5 *kg FM compost/kg DM compost = kg FM compost/ha yr
elif scen == 'S0_Ammon_salts':
    activity["Compost"]  = Phosphorus_Balance['P_Compost']*kg_compostFM_kg_P2O5*kg_DMcompost_kg_FMcompost #kg P2O5/ha yr * kg DMcompost/kg P2O5 *kg FM compost/kg DM compost = kg FM compost/ha yr [should be zeros!!]
else:
    pass

In [ ]:
N_fert_total = np.sum(activity['Area']*(1/10000) * activity["N_Fertilizer"])/1000
N_fert_total

In [ ]:
N_ammonSalts_total = np.sum(activity['Area']*(1/10000) * activity["N_Ammonium_salts"])/1000
N_ammonSalts_total

In [ ]:
P_fert_total = np.sum(activity['Area']*(1/10000) * activity["P_Fertilizer"])/1000
P_fert_total

In [ ]:
K_fert_total = np.sum(activity['Area']*(1/10000) * activity["K_Fertilizer"])/1000
K_fert_total

In [ ]:
Struvite_total = np.sum(activity['Area']*(1/10000) * activity["Struvite"])/1000
Struvite_total

In [ ]:
Compost_total = np.sum(activity['Area']*(1/10000) * activity["Compost"])/1000
Compost_total

In [ ]:
#Check cell, here we check that the total compost calculated does not exceed the 2016 yearly production of the three plants
#Total compost production in three plant from AMB in 2016 is 5,016.65 kg FM compost/yr
#the difference with what we calculate as the total of compost here assuming weighted averages is
#the total should be positive otherwise we are exceeding the production
5016.65 - Compost_total

In [ ]:
#Check cell, here we check that the total ammonium salts calculated does not exceed the yearly production of llobregat
#Total Ammonium sulphur production is 2816.30 kg/yr
#the difference with what we calculate as the total
#the total should be positive otherwise we are exceeding the production
2816.30 - N_ammonSalts_total

In [ ]:
#Exploring the activity values via plot fid
activity.loc[(activity['Fid']==2544)]

In [ ]:
#calculating background impact for each database and product
#Multiplying impact per kg of product (e.g. kg CO2eq/kg N) with activity i.e. N,P,K mineral fert and struvite input (e.g. kg N/ha yr) = kg CO2eq/ha yr 
LCIA = pd.DataFrame()
for database in databases:
    for impact in impacts:
        for product in products:
            CF = ReCiPe_LCIA.loc[(ReCiPe_LCIA['database']==database)&
                                     (ReCiPe_LCIA['impact']==impact)&
                                     (ReCiPe_LCIA['reference product']==product)]['value']
            URBAG_map['database']=database
            URBAG_map['imp_cat']=impact
            URBAG_map['product']=product
            if product=='inorganic nitrogen fertiliser, as N':
                URBAG_map['imp_per_ha'] = activity["N_Fertilizer"].mul(CF, fill_value=CF)
            elif product=='inorganic phosphorus fertiliser, as P2O5':
                URBAG_map['imp_per_ha'] = activity["P_Fertilizer"].mul(CF, fill_value=CF)
            elif product=='inorganic potassium fertiliser, as K2O':
                URBAG_map['imp_per_ha'] = activity["K_Fertilizer"].mul(CF, fill_value=CF)
            elif product=='Struvite at WWTP':
                URBAG_map['imp_per_ha'] = activity["Struvite"].mul(CF, fill_value=CF)
            elif product=='compost, market for compost Metropolitan Area of Barcelona':
                URBAG_map['imp_per_ha'] = activity["Compost"].mul(CF, fill_value=CF)
            elif product=='Ammonium sulphate at WWTP':
                URBAG_map['imp_per_ha'] = activity["N_Ammonium_salts"].mul(CF, fill_value=CF)
            else:
                pass
            LCIA = pd.concat([LCIA,URBAG_map])

In [ ]:
LCIA.columns.values

In [ ]:
LCIA.loc[LCIA["product"]=="inorganic nitrogen fertiliser, as N"].head()

In [ ]:
pd.unique(LCIA["database"])

In [ ]:
#Adding total impacts per year
#Total impact per parcel in kg impact units/yr 
#!!!!!!conveting to tons>> GWP = Ton CO2eq; MEP= Ton Neq; FEP= Ton P eq; WD = *1000 m3!!!!!
#Adding background label
#Adding scenario scen
LCIA['imp_yr']=(LCIA['imp_per_ha']*LCIA['Area']*(1/10000))/1000 #ton impact/yr
LCIA['LCIA']  ='background'
LCIA['Scenario'] =scen

In [ ]:
LCIA.head()

In [ ]:
#plots*product*impacts*databases
#S4=11334
#S0=4581
11334*6*5*3

In [ ]:
4581*6

In [ ]:
LCIA.columns

In [ ]:
LCIA_ei391 = LCIA[LCIA["database"]=="ei391_cutoff"]
LCIA_ei391_GWP = LCIA_ei391[LCIA_ei391["imp_cat"]=="ReCiPe 2016 v1.03, midpoint (H) | water use | water consumption potential (WCP)"]
LCIA_ei391_GWP[LCIA_ei391_GWP["product"]=="inorganic nitrogen fertiliser, as N"].imp_yr.sum()

In [ ]:
#Changing long databases names for short names
database_short = {
       'ei391': ['ei391_cutoff'],
        #'ei38_SSP2-Base_30':['ei38_cutoff_IMAGE_SSP2Base_2030'],
        #'ei38_SSP2-Base_40':['ei38_cutoff_IMAGE_SSP2Base_2040'],
        'ei391_Image_SSP2_Base_50':['ei391_cutoff_image_SSP2_Base_2050'],
        #'ei38_SSP2-RCP2.6_30':['ei38_cutoff_IMAGE_SSP2RCP26_2030'],
        #'ei38_SSP2-RCP2.6_40':['ei38_cutoff_IMAGE_SSP2RCP26_2040'],
        'ei391_Image_SSP2_RCP26_50':['ei391_cutoff_image_SSP2_RCP26_2050']}

In [ ]:
imp_cat_short = {'GWP100': ['IPCC 2021 | climate change | global warming potential (GWP100)'],
       'FE':['ReCiPe 2016 v1.03, midpoint (H) | eutrophication: freshwater | freshwater eutrophication potential (FEP)'],
       'ME':[ 'ReCiPe 2016 v1.03, midpoint (H) | eutrophication: marine | marine eutrophication potential (MEP)'],
       'WC':['ReCiPe 2016 v1.03, midpoint (H) | water use | water consumption potential (WCP)'],
       'ADP':['CML v4.8 2016 | material resources: metals/minerals | abiotic depletion potential (ADP): elements (ultimate reserves)']}

In [ ]:
#Adding a column with short names for impact category
imp_cat_shorter = {}
for key, impac in imp_cat_short.items():
    
    for x in impac:
        imp_cat_shorter[x] = key

LCIA['imp_cat2'] = LCIA['imp_cat'].map(imp_cat_shorter)

pd.unique(LCIA['imp_cat2'])

In [ ]:
#Adding a column with short names for databases
database_shorter = {}
for key, db in database_short.items():
    
    for x in db:
        database_shorter[x] = key

LCIA['database2'] = LCIA['database'].map(database_shorter)

pd.unique(LCIA['database2'])

In [ ]:
#Dropping long names for impact categories and databases
LCIA = LCIA.drop(['imp_cat','database'], axis=1)

In [ ]:
#Changing the column name for short impact names to match foreground/background impacts too
LCIA.rename(columns={'imp_cat2': 'imp_cat', 'database2': 'database'}, inplace=True)

In [ ]:
pd.unique(LCIA['database'])

In [ ]:
#Exploring the list of impacts via plot fid, database, impact, product
LCIA.loc[#(LCIA['Fid']==2544)
                   (LCIA['database']=='ei391')
                   &(LCIA['imp_cat']=='WC')
                   &(LCIA['product']=='inorganic nitrogen fertiliser, as N')
                  ].imp_yr.sum()

In [ ]:
#Total impact in ton impact unit/yr for the whole AMB, per product and database and scenario
LCIA_all_1 = LCIA.groupby(["LCIA",'database', "product", "Scenario","imp_cat"])["imp_yr"].aggregate("sum").reset_index()
LCIA_all_1

In [ ]:
LCIA_all_1[LCIA_all_1["database"]=="ei391"]

In [ ]:
#copying background impacts for regionalized ME and FE. These are not regionalized
FE_labels = ["FE","FE_Reg"]

In [ ]:
#copying background impacts for regionalized ME and FE. These are not regionalized
ME_labels = ["ME","ME_Reg"]

In [ ]:
#duplicate rows for FE and ME background impacts as these are the same for the regionalized foreground impacts and the recipe foreground impacts
duplicated_rows = LCIA_all_1[LCIA_all_1['imp_cat'].isin(['FE', 'ME'])].copy()

In [ ]:
duplicated_rows['imp_cat'] = duplicated_rows['imp_cat'].map({'FE': 'FE_Reg', 'ME': 'ME_Reg'})

In [ ]:
duplicated_rows

In [ ]:
LCIA_all = pd.concat([LCIA_all_1, duplicated_rows], ignore_index=True)

In [ ]:
LCIA_all[LCIA_all["database"]=="ei391"]

In [ ]:
#checking the totals for one impact remain the same as in the LCIA variable
test = LCIA_all[LCIA_all["database"]=="ei391"]
test[test["imp_cat"]=="WC"]

In [ ]:
pd.unique(LCIA_all["imp_cat"])

In [ ]:
LCIA_all

In [ ]:
#Exploring the list of impacts via plot fid, database, impact, product
LCIA_all.loc[(LCIA_all['database']=='ei391')
                   &(LCIA_all['imp_cat']=='GWP100')
                   &(LCIA_all['product']=='compost, market for compost Metropolitan Area of Barcelona')
                  ].imp_yr.sum()

In [ ]:
#Separate Total impacts in ton impact unit/yr for the whole AMB, per product and database and scenario
LCIA_GWP = LCIA_all.loc[LCIA_all["imp_cat"]=="GWP100"].drop(["imp_cat"],axis=1)
LCIA_ME = LCIA_all.loc[LCIA_all["imp_cat"]=="ME"].drop(["imp_cat"],axis=1)
LCIA_FE = LCIA_all.loc[LCIA_all["imp_cat"]=="FE"].drop(["imp_cat"],axis=1)
LCIA_WD  = LCIA_all.loc[LCIA_all["imp_cat"]=="WD"].drop(["imp_cat"],axis=1)
LCIA_ADP  = LCIA_all.loc[LCIA_all["imp_cat"]=="ADP"].drop(["imp_cat"],axis=1)

In [ ]:
LCIA_GWP.loc[LCIA_GWP["product"]=="compost, market for compost Metropolitan Area of Barcelona"]

In [ ]:
URBAG_map.keys()

In [ ]:
URBAG_map = URBAG_map.drop(['database'],axis=1)
URBAG_map

In [ ]:
URBAG_map.keys()

## Visualization  

In [ ]:
#ton impact units/yr
data_back     = LCIA_all
data_back_GWP = LCIA_GWP
data_back_ME  = LCIA_ME
data_back_FE  = LCIA_FE
data_back_WD  = LCIA_WD
data_back_ADP  = LCIA_ADP

In [ ]:
pd.unique(data_back["imp_cat"])

In [ ]:
pd.unique(data_back['Scenario'])

In [ ]:
pd.unique(data_back['imp_cat'])

In [ ]:
pd.unique(data_back['product'])

In [ ]:
data_back.keys()

In [ ]:
data_back

In [ ]:
data_back_GWP

## Saving outputs

In [ ]:
%store data_back
%store URBAG_map

In [ ]:
#Total (prospective) background impacts in tons impacts per yr
data_back.to_csv("output/" + scen + "/LCIA/backgroundImpacts/Total_back_impacts.csv")

In [ ]:
#Map background impacts full dataframe
#imp_per_ha in kg or m3 per ha per yr
#imp_yr in tons or *1000m3 per year
LCIA.to_csv("output/" + scen + "/LCIA/backgroundImpacts/Map_back_impacts.csv")